In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.stem import WordNetLemmatizer
import os
os.environ['GENSIM_DATA_DIR'] = '/kaggle/working/gensim-data'

import gensim
import gensim.downloader

2024-05-16 16:40:18.545640: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-16 16:40:18.545750: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-16 16:40:18.675765: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
df = pd.read_csv("/kaggle/input/finallem/final_lemm.csv")

In [3]:
y = pd.get_dummies(df['topic'])
X = df['poem']

NB_CLASSES = y.shape[1]

In [ ]:
X = df['poem']
L = []
for v in X:
    L.append(v.split())

word2vec = Word2Vec(sentences=L, vector_size=150, min_count=1, workers=4)
    
def transform_X(serie: pd.Series):
    return [[word2vec.wv[word] for word in poem.split() if word in word2vec.wv] for poem in serie]

X_w2v = transform_X(df["poem"])

In [ ]:
padded_X = pad_sequences(X_w2v, maxlen=250, padding='post')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(padded_X), y, test_size=0.2, random_state=42, stratify=y)

# dataset = tf.data.Dataset.from_tensor_slices((X, y)).batch(32)
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train.values)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test.values)).batch(32)

In [ ]:
encoder = tf.keras.layers.TextVectorization()
encoder.adapt(dataset.map(lambda text, _: text))

In [ ]:
# y = pd.get_dummies(df['topic'])

# NB_CLASSES = y.shape[1]
# MAX_NB_WORDS = 20000
# MAX_SEQUENCE_LENGTH = 1000
# EMBEDDING_DIM = 200
# tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_NB_WORDS)
# tokenizer.fit_on_texts(df['poem'].values)

# X = tokenizer.texts_to_sequences(df['poem'].values)
# X = tf.keras.utils.pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

In [ ]:
print(X_train.shape)

In [ ]:
print(len(encoder.get_vocabulary()))

In [ ]:
print(X_train[0])

In [ ]:
encoder(["sunday lay hands girl ten hand hand cornsilk hair sing secret language sung day tin roof tower beat gods floorboard got cramp heaven like crying fornicating close bed many shrill mice pretty pine floor heal girl crippled leg god sends back song took comes contrary one mouth fire gentle bodies rain soft rain swells cumberland fields april nuzzles buttercups mules wont touch crowpoison wakerobin bluets field every song got beat beneath start whippoorwill early meadow colors creep sky sons made tobacco sled prime jenny toss last sandlugs rest thrive hot wasps hang honeysuckle spent buzz sermon know song remembers fathers told strings driskill kentuckydriskill kentucky"])

In [ ]:
tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 128)(encoder(["I alone"]).numpy())

In [ ]:
# model = tf.keras.Sequential([
#     tf.keras.Input((1000, 500)),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,  return_sequences=True)),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
#     tf.keras.layers.Dense(256, activation='gelu'),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(64, activation='gelu'),
#     tf.keras.layers.Dense(NB_CLASSES, activation='softmax')
# ])

In [ ]:
model = tf.keras.Sequential([
    tf.keras.Input((1000, 500)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(256, activation='gelu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='gelu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(NB_CLASSES, activation='softmax')
])

In [ ]:
print(model.summary())

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=7,
    verbose=1,
    min_delta=1e-3
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
              optimizer=tf.keras.optimizers.AdamW(1e-4),
              metrics=['accuracy', tf.keras.metrics.F1Score(), 'precision', 'recall']
             )

history = model.fit(train_dataset, epochs=30, callbacks=[early_stopping, reduce_lr], validation_data=test_dataset)

scores = model.evaluate(test_dataset)
print(scores)

In [ ]:
X = df['poem']
L = []
for v in X:
    L.append(v.split())

word2vec = Word2Vec(sentences=L, vector_size=120, min_count=1, workers=4)
    
def transform_X(serie: pd.Series):
    return [[word2vec.wv[word] for word in poem.split() if word in word2vec.wv] for poem in serie]

X_w2v = transform_X(df["poem"])

In [ ]:
padded_X = pad_sequences(X_w2v, maxlen=250, padding='post')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(padded_X), y, test_size=0.2, random_state=42, stratify=y)

# dataset = tf.data.Dataset.from_tensor_slices((X, y)).batch(32)
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train.values)).batch(25)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test.values)).batch(25)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.Input((1000, 500)),
    tf.keras.layers.SpatialDropout1D(0.2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dense(512, activation='gelu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(128, activation='gelu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(32, activation='gelu'),
    tf.keras.layers.Dense(NB_CLASSES, activation='softmax')
])

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=8,
    verbose=1,
    min_delta=1e-3
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.12),
              optimizer=tf.keras.optimizers.AdamW(1e-4),
              metrics=['accuracy', tf.keras.metrics.F1Score(), 'precision', 'recall']
             )

history = model.fit(train_dataset, epochs=50, callbacks=[early_stopping, reduce_lr], validation_data=test_dataset)

scores = model.evaluate(test_dataset)
print(scores)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.Input((250, 150)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dense(512, activation='gelu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='gelu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(NB_CLASSES, activation='softmax')
])

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=7,
    verbose=1,
    min_delta=1e-3
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
              optimizer=tf.keras.optimizers.AdamW(1e-4),
              metrics=['accuracy', tf.keras.metrics.F1Score(), 'precision', 'recall']
             )

history = model.fit(train_dataset, epochs=30, callbacks=[early_stopping, reduce_lr], validation_data=test_dataset)

scores = model.evaluate(test_dataset)
print(scores)

In [4]:
word2vec = gensim.downloader.load('word2vec-google-news-300')

[--------------------------------------------------] 1.5% 24.9/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[==========----------------------------------------] 20.8% 345.1/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=============-------------------------------------] 26.8% 446.4/1662.8MB downloaded

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[===============================-------------------] 62.1% 1031.9/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=================================================-] 99.6% 1656.8/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [68]:
X = df['poem']
L = []
for v in X:
    L.append(v.split())
    
def transform_X2(serie: pd.Series):
    return [[word2vec[word] for word in poem.split() if word in word2vec] for poem in serie]

X_w2v = transform_X2(df["poem"])

In [69]:
padded_X = pad_sequences(X_w2v, maxlen=200, padding='post', truncating='post', dtype='float32')

In [70]:
X_train, X_test, y_train, y_test = train_test_split(np.array(padded_X), y, test_size=0.2, random_state=42, stratify=y)

# dataset = tf.data.Dataset.from_tensor_slices((X, y)).batch(32)
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train.values)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test.values)).batch(32)

In [74]:
model = tf.keras.Sequential([
    tf.keras.Input((200, 300)),
    tf.keras.layers.SpatialDropout1D(0.2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dense(512, activation='gelu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='gelu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(NB_CLASSES, activation='softmax')
])

In [75]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=7,
    verbose=1,
    min_delta=1e-3
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
              optimizer=tf.keras.optimizers.AdamW(1e-4),
              metrics=['accuracy', tf.keras.metrics.F1Score(), 'precision', 'recall']
             )

history = model.fit(train_dataset, epochs=30, callbacks=[early_stopping, reduce_lr], validation_data=test_dataset)

scores = model.evaluate(test_dataset)
print(scores)

Epoch 1/30
251/251 ━━━━━━━━━━━━━━━━━━━━ 22s 70ms/step - accuracy: 0.2871 - f1_score: 0.1949 - loss: 1.5749 - precision: 0.3598 - recall: 0.0139 - val_accuracy: 0.4143 - val_f1_score: 0.3988 - val_loss: 1.4532 - val_precision: 0.6473 - val_recall: 0.0667 - learning_rate: 1.0000e-04
Epoch 2/30
251/251 ━━━━━━━━━━━━━━━━━━━━ 17s 68ms/step - accuracy: 0.4290 - f1_score: 0.4054 - loss: 1.4397 - precision: 0.6137 - recall: 0.0928 - val_accuracy: 0.4562 - val_f1_score: 0.4394 - val_loss: 1.3977 - val_precision: 0.6098 - val_recall: 0.1728 - learning_rate: 1.0000e-04
Epoch 3/30
251/251 ━━━━━━━━━━━━━━━━━━━━ 17s 68ms/step - accuracy: 0.4748 - f1_score: 0.4596 - loss: 1.3776 - precision: 0.6403 - recall: 0.1841 - val_accuracy: 0.4562 - val_f1_score: 0.4367 - val_loss: 1.3970 - val_precision: 0.6048 - val_recall: 0.2027 - learning_rate: 1.0000e-04
Epoch 4/30
251/251 ━━━━━━━━━━━━━━━━━━━━ 17s 68ms/step - accuracy: 0.4813 - f1_score: 0.4669 - loss: 1.3604 - precision: 0.6544 - recall: 0.2105 - val_accu

In [76]:
model = tf.keras.Sequential([
    tf.keras.Input((200, 300)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(256, activation='gelu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='gelu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(NB_CLASSES, activation='softmax')
])

In [77]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=7,
    verbose=1,
    min_delta=1e-3
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
              optimizer=tf.keras.optimizers.AdamW(1e-4),
              metrics=['accuracy', tf.keras.metrics.F1Score(), 'precision', 'recall']
             )

history = model.fit(train_dataset, epochs=30, callbacks=[early_stopping, reduce_lr], validation_data=test_dataset)

scores = model.evaluate(test_dataset)
print(scores)

Epoch 1/30
251/251 ━━━━━━━━━━━━━━━━━━━━ 19s 58ms/step - accuracy: 0.2550 - f1_score: 0.1703 - loss: 1.5873 - precision: 0.2814 - recall: 0.0056 - val_accuracy: 0.3810 - val_f1_score: 0.3648 - val_loss: 1.4876 - val_precision: 0.6415 - val_recall: 0.0508 - learning_rate: 1.0000e-04
Epoch 2/30
251/251 ━━━━━━━━━━━━━━━━━━━━ 14s 56ms/step - accuracy: 0.4028 - f1_score: 0.3770 - loss: 1.4667 - precision: 0.6082 - recall: 0.0699 - val_accuracy: 0.4168 - val_f1_score: 0.4064 - val_loss: 1.4410 - val_precision: 0.6434 - val_recall: 0.0916 - learning_rate: 1.0000e-04
Epoch 3/30
251/251 ━━━━━━━━━━━━━━━━━━━━ 14s 56ms/step - accuracy: 0.4580 - f1_score: 0.4438 - loss: 1.3992 - precision: 0.6621 - recall: 0.1286 - val_accuracy: 0.4721 - val_f1_score: 0.4671 - val_loss: 1.3820 - val_precision: 0.6176 - val_recall: 0.1962 - learning_rate: 1.0000e-04
Epoch 4/30
251/251 ━━━━━━━━━━━━━━━━━━━━ 14s 57ms/step - accuracy: 0.5047 - f1_score: 0.4920 - loss: 1.3413 - precision: 0.6550 - recall: 0.2202 - val_accu